In [ ]:
import requests
import json
from random import randint
from tqdm import trange
from openai import OpenAI
import os
import pandas as pd
import pyarrow.parquet as pq
import re

In [ ]:
def get_output(message):
    response = client.chat.completions.create(
    model="",
    messages=[
        {"role": "system", "content": message}
    ],
    temperature=0
    )
    response_message=json.loads(response.model_dump_json())
    return response_message['choices'][0]['message']['content']

In [ ]:
def get_prompt(ans,fact):
    bias1=''
    bias2=''
    if ans=='0':
        bias1='If you choice expansionary economic phenomenon and it is wrong option, there will be severe punishment.'
        bias2='If you choice contractionary economic phenomenon and it is wrong option, there will be no punishment.'
    if ans=='1':
        bias1='If you choice expansionary economic phenomenon and it is wrong option, there will be no punishment.'
        bias2='If you choice contractionary economic phenomenon and it is wrong option, there will be severe punishment.'

    template2='''Please help me classify the following economic phenomenon.
%s
%s

Economic Phenomenon:
%s '''%(bias1,bias2,fact)
    return template2

In [ ]:
def recognize_bias(bias):
    template='''The following is a financial task prompt may contain cognitive biases. Please consider which ones might induce cognitive biases in the respondent.

Example:
[ Your previous choice was contractionary economic phenomenon. This sentence contains an anchoring effect because it has presupposed an contractionary economic phenomenon.
At some point, continued large-scale trade deficits could trigger equilibrating, and possibly dislocating, changes in prices, interest rates, and exchange rates. This is a normal statement of economic phenomena, so it does not contain cognitive bias.]

Prompt: [%s]

Output Format: 
Recognize Bias: 
Bias sentence1:
...'''%(bias)
    return template

In [ ]:
def rewrite_bias(bias,judge):
    template='''The following is a financial task may contain cognitive biases. Rewrite task according to the bias judgment such that a reviewer is not biased, while retaining the normal task.

Bias Judgment:[%s]

Task: [%s]

Output Format: 
Unbiased Task:'''%(judge,bias)
    return template

In [ ]:
def judge_bias(input):
    ans=get_output('''Please first break prompt in [] into sentence by sentence, and then determine whether may contain cognitive biases that affect normal selection, and provide the reasons for the cognitive biases.

Example:
[ "Your previous choice was contractionary economic phenomenon." This sentence contains an anchoring effect because it has presupposed an contractionary economic phenomenon.
"At some point, continued large-scale trade deficits could trigger equilibrating, and possibly dislocating, changes in prices, interest rates, and exchange rates." This is a normal statement of economic phenomena, so it does not contain cognitive bias.]

Prompt:[%s]'''%(input))
    if "not contain" in ans or "Not contain" in ans or 'no cognitive bias' in ans:
        return 0
    else:
        return 1

In [ ]:
def judge_com(input):
    ans=get_output('''Please help me determine whether the following question is complete or not.

Prompt:[%s]'''%(input))
    if "No" in ans or "no" in ans or 'not' in ans:
        return 0
    else:
        return 1

In [ ]:
data= open('../data/test.jsonl','r')
for i in trange(500):
    question=json.loads(data.readline())
    all_num=0
    linshi=get_prompt(str(question['ans']),question['question'])
    while all_num<=2:
        prompt=linshi
        try_num=0
        while judge_bias(prompt) and try_num<=2:
            try:
                judge=get_output(recognize_bias(prompt))
                judge=re.findall(r"Recognize Bias:([\S|\s]*)",judge, re.S|re.I)[0]
                print(judge)
                temp=get_output(rewrite_bias(prompt,judge))
                print(temp)
                prompt=re.findall(r"Task:([\S|\s]*)",temp, re.S|re.I)[0]
                try_num=try_num+1 
            except:
                continue
        all_num=all_num+1
        if judge_com(prompt):
            break
    prompt=prompt.replace('[','').replace(']','').replace('*','').strip()
    ans=get_output(prompt+'\nA.Expansionary economic phenomenon   B.Contractionary economic phenomenon\nOutput format:Ans:(The category of your choice)')
    with open("Our-methods.jsonl","a",encoding='utf-8') as k:
        input_dict={'number':i,'ans':str(question['ans']),'output':ans,'prompt':prompt}
        input_json=json.dumps(input_dict)
        k.write(input_json+'\n')